In [2]:
import tensorflow as tf
import qkera_experiments.dataset as dataset
from tensorflow import keras
from keras import layers
from qkeras import *

# load the dataset

In [3]:
x_train, y_train, x_test, y_test = dataset.load_cifar10()

(50000, 1)


# load the model

In [4]:
from qkeras.utils import load_qmodel


qmodel = load_qmodel("C:/Users/georg/Desktop/mestrado_luiz/deep_nibble_dnn/qkera_experiments/qmodels/resnet32_hybrid/model.h5")
qmodel.evaluate(x_test, y_test)

x_in = qmodel.layers[0].input
x_out = qmodel.layers[-2].output

base_model = keras.Model(inputs=[x_in], outputs=[x_out])
base_model.summary()

del qmodel

c:\Users\georg\Desktop\mestrado_luiz\deep_nibble_dnn\.venv\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


313/313 [==============================] - 13s 28ms/step - loss: 1.1779 - accuracy: 0.7028
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 34, 34, 3)    0           ['input[0][0]']                  
                                                                                                  
 qconv1 (Conv2D)                (None, 32, 32, 16)   432         ['conv1_pad[0][0]']              
                                                                                                  
 bn1 (BatchNormalization)       (None, 32, 32, 16)   64          ['qconv1[0][0]']                 
   

# including base model as a preprocessing component of the Deep Nibble MLP training

In [5]:
# preprocess the dataset
x_train_2 = base_model.predict(x_train)
print(x_train_2.shape)

1563/1563 [==============================] - 45s 27ms/step
(50000, 64)


In [6]:
# preprocess the dataset
x_test_2 = base_model.predict(x_test)
print(x_test_2.shape)

313/313 [==============================] - 8s 26ms/step
(10000, 64)


In [7]:
del base_model
del x_test
del x_train

In [20]:
import numpy as np
import tensorflow as tf
from mlp_from_scratch.NeuralNetwork import QNeuralNetworkWithScale


# Define neural network inputnoutput
input_size = x_train_2.shape[1]
output_size = y_train.shape[1]

print(x_train_2.shape,
x_test_2.shape,
y_train.shape,
y_test.shape)

# Create and train the neural network
neural_network_with_scale = QNeuralNetworkWithScale(input_size, output_size)

(50000, 64) (10000, 64) (50000, 10) (10000, 10)


In [21]:
# train the nn
neural_network_with_scale.train(x_train_2, y_train, learning_rate=0.000010, num_epochs=1, batch_size=256, x_val=x_test_2, y_val=y_test)
neural_network_with_scale.train(x_train_2, y_train, learning_rate=0.000100, num_epochs=7, batch_size=256, x_val=x_test_2, y_val=y_test)


Epoch 1/1, Loss: 0.007692290004342794 Accuracy: 39.18000030517578%
Epoch 1/7, Loss: 0.005479162558913231 Accuracy: 27.93000030517578%
Epoch 2/7, Loss: 0.006831869948655367 Accuracy: 28.099998474121094%
Epoch 3/7, Loss: 0.009056954644620419 Accuracy: 48.27000045776367%
Epoch 4/7, Loss: 0.005756912752985954 Accuracy: 64.54000091552734%
Epoch 5/7, Loss: 0.0024021996650844812 Accuracy: 63.709999084472656%
Epoch 6/7, Loss: 0.0021660919301211834 Accuracy: 66.80999755859375%
Epoch 7/7, Loss: 0.0028899225872009993 Accuracy: 66.18999481201172%
